# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("src/models")
from typing import Tuple
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier

# Import other libraries if needed

## Import Dataset

In [2]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

# df_train = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')

df_additional_features_test = pd.read_csv('data/test/additional_features_test.csv')
df_basic_features_test = pd.read_csv('data/test/basic_features_test.csv')
df_content_features_test = pd.read_csv('data/test/content_features_test.csv')
df_flow_features_test = pd.read_csv('data/test/flow_features_test.csv')
df_time_features_test = pd.read_csv('data/test/time_features_test.csv')

test = pd.merge(df_additional_features_test, df_basic_features_test, on='id')
test = pd.merge(test, df_basic_features_test, on='id')
test = pd.merge(test, df_content_features_test, on='id')
test = pd.merge(test, df_flow_features_test, on='id')
test = pd.merge(test, df_time_features_test, on='id')

test.head()

df_additional_features_train = pd.read_csv('data/train/additional_features_train.csv')
df_labels_train = pd.read_csv('data/train/labels_train.csv')
df_basic_features_train = pd.read_csv('data/train/basic_features_train.csv')
df_content_features_train = pd.read_csv('data/train/content_features_train.csv')
df_flow_features_train = pd.read_csv('data/train/flow_features_train.csv')
df_time_features_train = pd.read_csv('data/train/time_features_train.csv')

train = pd.merge(df_additional_features_train, df_basic_features_train, on='id')
train = pd.merge(train, df_basic_features_train, on='id')
train = pd.merge(train, df_labels_train, on='id')
train = pd.merge(train, df_content_features_train, on='id')
train = pd.merge(train, df_flow_features_train, on='id')
train = pd.merge(train, df_time_features_train, on='id')



In [3]:
print(len(train))
print(len(test))

175341
20583


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [4]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
train_set, val_set = train_test_split(train, test_size= 0.33, random_state= 42)

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [5]:
# Write your code here
class FeatureImputer(BaseEstimator, TransformerMixin):
    """Handles missing values based on thresholds"""
    def __init__(self, column_threshold=0.5, row_threshold=5):
        self.column_threshold = column_threshold
        self.row_threshold = row_threshold
        self.columns_to_drop = None
        self.mode_values = None
        self.mean_values = None
        self.replacement_values = None

    def fit(self, X, y=None):
        # Drop columns with too many nulls
        null_counts = X.isna().sum()

        self.columns_to_drop = null_counts[null_counts > len(X) * self.column_threshold].index

        # Store mode values from training data
        X_temp = X.drop(columns=self.columns_to_drop)

        # For categorical columns, use mode for categorical and mean for numerical
        self.mode_values = X_temp.select_dtypes(include=['object']).mode().iloc[0]
        self.mean_values = X_temp.select_dtypes(include=['int64', 'float64']).mean()
        self.replacement_values = pd.concat([self.mode_values, self.mean_values])

        return self

    def transform(self, X):

        X_transformed = X.copy()

        # Drop identified columns
        if self.columns_to_drop is not None:
            X_transformed = X_transformed.drop(columns=self.columns_to_drop)

        # Fill nulls with replacement values
        null_counts_before = X_transformed.isna().sum()
        X_transformed = X_transformed.fillna(self.replacement_values)
        null_counts_after = X_transformed.isna().sum()

        return X_transformed

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [6]:
# Write your code here
# handling outlier using clipping method
# and checking it using iqr
class OutlierImputer(BaseEstimator, TransformerMixin):
    """Handles outliers in numerical features based on statistical thresholds."""
    def __init__(self, threshold=1.5):
        self.threshold = threshold
        self.feature_limits = None

    def fit(self, X, y=None):
        self.feature_limits = {}

        for col in X.select_dtypes(include=['int64', 'float64']).columns:

            q1 = X[col].quantile(0.25)
            q3 = X[col].quantile(0.75)
            iqr = q3 - q1
            lower_limit = q1 - self.threshold * iqr
            upper_limit = q3 + self.threshold * iqr

            self.feature_limits[col] = (lower_limit, upper_limit)

        return self

    def transform(self, X):
        X_transformed = X.copy()

        for col, (lower, upper) in self.feature_limits.items():
            X_transformed[col] = X_transformed[col].clip(lower=lower, upper=upper)

        return X_transformed

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [7]:
# Write your code here
# handling duplicate data
class DuplicateImputer(BaseEstimator, TransformerMixin):
    """Handles duplicate data by checking ID column."""

    def fit(self, X):
        # do nothing
        return self

    def transform(self, X):
        X_transformed = X.drop_duplicates()
        return X_transformed

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

removing id column, it'll be done later

In [8]:
# Write your code here

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [9]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [10]:
# Write your code here
# min max scaling

class FeatureScaler(BaseEstimator, TransformerMixin):
    """
    Scales features using min-max normalization
    """

    def __init__(self, feature_range=(0, 1), columns=None, exclude_columns=None):
        """
        Initialize the FeatureScaler.
        """
        self.feature_range = feature_range
        self.columns = columns
        self.exclude_columns = exclude_columns or []
        self.scaled_columns = None
        self.scaling_info = None

    def fit(self, X, y=None):
        """
        Compute the minimum and maximum values for scaling.
        """

        # Create a copy of the input DataFrame
        X_copy = X.copy()

        numeric_columns = X_copy.select_dtypes(include=['int64', 'float64']).columns
        self.scaled_columns = [
            col for col in numeric_columns 
            if col not in self.exclude_columns
        ]

        # Prepare scaling information
        self.scaling_info = {}

        # Compute min and max for each selected column
        for col in self.scaled_columns:
            original_min = X_copy[col].min()
            original_max = X_copy[col].max()

            self.scaling_info[col] = {
                'original_min': original_min,
                'original_max': original_max,
            }

        return self

    def transform(self, X):
        """
        Scale features using the computed minimum and maximum values.

        Parameters:
        -----------
        X : pandas.DataFrame
            Input data to transform

        Returns:
        --------
        pandas.DataFrame
            Transformed data with scaled features
        """
        if self.scaling_info is None:
            raise ValueError("Scaler has not been fitted. Call fit() first.")

        # Create a copy of the input DataFrame
        X_transformed = X.copy()

        # Scale the selected columns
        for col in self.scaled_columns:
            scaling_details = self.scaling_info[col]
            original_min = scaling_details['original_min']
            original_max = scaling_details['original_max']

            # Apply min-max scaling formula
            if(original_min == original_max):
              X_transformed[col] = 0.5
            else:
              X_transformed[col] = (X_transformed[col] - original_min) / (original_max - original_min)

        return X_transformed

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [11]:
# Write your code here
class FeatureEncoder(BaseEstimator, TransformerMixin):
    """Handles categorical variables using OneHotEncoder"""
    def __init__(self):
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.categorical_columns = None
        self.feature_names = None

    def fit(self, X, y=None):

        # Identify categorical columns
        self.categorical_columns = [
            col for col in X.select_dtypes(include=['object']).columns
        ]

        # categorical value to numerical
        if self.categorical_columns:
            self.encoder.fit(X[self.categorical_columns])

            # Store feature names
            self.feature_names = [
                f"{col}_{val}" for col, vals in zip(
                    self.categorical_columns,
                    self.encoder.categories_
                ) for val in vals
            ]

        return self

    def transform(self, X):

        X_transformed = X.copy()

        if self.categorical_columns:
            encoded_features = self.encoder.transform(X[self.categorical_columns])

            # Create dataframe with encoded features
            encoded_df = pd.DataFrame(
                encoded_features,
                columns=self.feature_names,
                index=X.index
            )

            # Drop original categorical columns and add encoded ones
            X_transformed = X_transformed.drop(columns=self.categorical_columns)
            X_transformed = pd.concat([X_transformed, encoded_df], axis=1)

        return X_transformed

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [12]:
"""Balances data by oversampling attack_cat using SMOTE."""
class ImbalancedDataHandler(BaseEstimator, TransformerMixin):
    def __init__(self, random_state=42):
        self.smote = SMOTE(sampling_strategy='auto', random_state=random_state)
        self.label_encoder = LabelEncoder()
    
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        """
        Balance the 'attack_cat' column using SMOTE.
        """
        if 'attack_cat' not in y.columns:
            raise ValueError("`y` must contain the 'attack_cat' column.")
        
        # Encode attack_cat to numeric values
        y_encoded = self.label_encoder.fit_transform(y['attack_cat'])
        
        # Apply smote and decode it back to attack_cat
        X_resampled, y_resampled = self.smote.fit_resample(X, y_encoded)
        y_resampled = pd.DataFrame({
            'attack_cat': self.label_encoder.inverse_transform(y_resampled)
        })
        
        X_resampled = pd.DataFrame(X_resampled, columns=X.columns)
        return X_resampled, y_resampled
    
    def fit_transform(self, X, y):
        return self.transform(X, y)

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

This process will reuse the feature scaler
so the feature scaler will be used twice to normalized pca result

In [13]:
# Write your code hereData already normalized during scaling


### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [14]:
# Write your code here
class PCAImputer(BaseEstimator, TransformerMixin):    
    def __init__(self, n_components=None, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.pca_ = None
    
    def fit(self, X, y=None):
        """
        Fit the PCA transformer.
        """

        # Validate input
        if X.isnull().any().any():
            raise ValueError("Input DataFrame contains null values.")
        
        # Determine number of components if not specified
        if self.n_components is None:
            full_pca = PCA(random_state=self.random_state)
            full_pca.fit(X)
            cumulative_variance_ratio = np.cumsum(full_pca.explained_variance_ratio_)
            self.n_components = np.argmax(cumulative_variance_ratio >= 0.85) + 1
        
        # Fit PCA
        self.pca_ = PCA(n_components=self.n_components, 
                        random_state=self.random_state)
        self.pca_.fit(X)
        
        return self
    
    def transform(self, X):
        """
        Transform input data using fitted PCA.
        """
        # Validate input
        if X.isnull().any().any():
            raise ValueError("Input DataFrame contains null values.")
        
        # Transform data
        X_transformed = self.pca_.transform(X)
        
        # Create DataFrame with generic column names
        reduced_columns = [f'PC_{i+1}' for i in range(X_transformed.shape[1])]
        return pd.DataFrame(X_transformed, 
                            columns=reduced_columns, 
                            index=X.index)
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [15]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [16]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [17]:
# Write your code here

# cleaning pipe
cleaning_pipe = Pipeline([("outlier", OutlierImputer()),
                 ("featureimputer", FeatureImputer())])


duplicateImputer = DuplicateImputer()
train_no_dupe = train.drop(['id'], axis = 1)
train_no_dupe = duplicateImputer.fit_transform(train_no_dupe)

# resplitting train data after removing duplicate
train_set, val_set = train_test_split(train_no_dupe, test_size= 0.33, random_state= 42)

# remove unrelated to classifying column
X_train_full = train_no_dupe.drop(['label','attack_cat'], axis=1)
X_train_init = train_set.drop(['label','attack_cat'], axis=1)
y_train_init = train_set[['label','attack_cat']]
X_val_init = val_set.drop(['label','attack_cat'], axis=1)
y_val_init = val_set[['label','attack_cat']]


X_train_full_cleaned = cleaning_pipe.fit_transform(X_train_full)
X_train_cleaned = cleaning_pipe.transform(X_train_init)
X_val_cleaned = cleaning_pipe.transform(X_val_init)

X_test = test.drop('id', axis=1)
X_test_cleaned = cleaning_pipe.transform(X_test)


C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance i

In [18]:
# preprocessing pipe

# pipe for x only data
pipe = Pipeline([("featureencoder", FeatureEncoder()),
                 ("imputer", FeatureScaler()),
                 ("pcaimputer", PCAImputer()),
                 ("normalizedimputer", FeatureScaler()),
                 ])

# fit using train full to avoid out of range
X_train_full_preprocessed = pipe.fit_transform(X_train_full_cleaned)
X_train_preprocessed = pipe.transform(X_train_cleaned)
X_val_preprocessed = pipe.transform(X_val_cleaned)
X_test_preprocessed = pipe.transform(X_test_cleaned)

# reset data index
# to make easier balancing
X_train_sorted = X_train_preprocessed.sort_index().reset_index(drop=True)
y_train_sorted = y_train_init.sort_index().reset_index(drop=True)
X_val_sorted = X_val_preprocessed.sort_index().reset_index(drop=True)
y_val_sorted = y_val_init.sort_index().reset_index(drop=True)

print(y_train_sorted['attack_cat'].value_counts())
print(y_train_sorted['label'].value_counts())

# handling imbalanced data
# separated because it uses the y
imbalanced_data_handler = ImbalancedDataHandler()
X_train_balanced, y_train_balanced = imbalanced_data_handler.fit_transform(X_train_sorted, y_train_sorted)
X_val_balanced, y_val_balanced = imbalanced_data_handler.fit_transform(X_val_sorted, y_val_sorted)

# All final results of the preprocessing step are renamed to maintain a consistent naming convention.
x_train_final = X_train_balanced
y_train_final = y_train_balanced
x_val_final = X_val_balanced
y_val_final = y_val_balanced
x_test_final = X_test_preprocessed

print("x_train_final =", x_train_final.head(10))
print("y_train_final =", y_train_final.head(10))
print("X_val_final =", x_val_final.head(10))
print("y_val_final =", y_val_final.head(10))
print("x_test_final =", x_test_final.head(10))

print(y_train_final['attack_cat'].value_counts())

# print(x_train_final.head(20))
# print(y_train_final.head(20))

C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance i

attack_cat
Normal            37389
Generic           24321
Exploits          21739
Fuzzers           12132
DoS                7877
Reconnaissance     6786
Analysis           1341
Backdoor           1177
Shellcode           769
Worms                90
Name: count, dtype: int64
label
1    76232
0    37389
Name: count, dtype: int64


C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  w

x_train_final =        PC_1      PC_2      PC_3      PC_4      PC_5      PC_6      PC_7  \
0  0.084739  0.559063  0.391656  0.422834  0.508430  0.409582  0.467690   
1  0.933277  0.305422  0.475992  0.101405  0.346156  0.681381  0.573132   
2  0.838367  0.756936  0.770207  0.216785  0.314822  0.523608  0.679156   
3  0.022473  0.626876  0.259850  0.275483  0.393395  0.348899  0.555942   
4  0.080963  0.563153  0.386316  0.408868  0.515396  0.399528  0.468708   
5  0.805970  0.293618  0.547440  0.139487  0.230065  0.555676  0.293777   
6  0.800613  0.925254  0.668334  0.319986  0.631802  0.379880  0.315335   
7  0.814510  0.855211  0.897694  0.215063  0.344813  0.295717  0.469572   
8  0.272972  0.239720  0.862276  0.441138  0.586616  0.526236  0.664715   
9  0.223061  0.288424  0.945458  0.385283  0.633666  0.364155  0.464461   

       PC_8      PC_9  
0  0.300923  0.398330  
1  0.382007  0.731120  
2  0.501684  0.526062  
3  0.403103  0.501488  
4  0.298442  0.400928  
5  0.438006  0

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [19]:
from KNN import KNN_FromScratch
from sklearn.metrics import accuracy_score
model = KNN_FromScratch(5, 2)
model.fit(x_train_final, y_train_final)
predictions = model.predict(x_val_final)
accuracy = accuracy_score(y_val_final, predictions)
print("Accuracy (Home):", accuracy)

model_scikit = KNeighborsClassifier(n_neighbors=5, p=2)
model_scikit.fit(x_train_final, y_train_final)
predictions_scikit = model_scikit.predict(x_val_final)
accuracy_scikit = accuracy_score(y_val_final, predictions_scikit)
print("Accuracy (Scikit):", accuracy_scikit)

Accuracy (Home): 0.675
Accuracy (Scikit): 0.67


C:\Users\devin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


## B. Naive Bayes

In [20]:
from src.models.NaiveBayes import GaussianNaiveBayes
from sklearn.metrics import accuracy_score

# Train the model
# model = GaussianNaiveBayes()

# print(X_train_sorted)

# model.fit(X_train_sorted, y_train_sorted['attack_cat'])
# model.save_model('NaiveBayesSorted.pkl')


# print(len(x_val_final))
# print(len(x_train_final))

# y_val_sorted = y_val_sorted['attack_cat']
model = GaussianNaiveBayes.load_model('NaiveBayesSorted.pkl')
prediction = model.predict(X_val_sorted)
accuracy = accuracy_score(y_val_sorted, prediction)
print("Accuracy: ", accuracy)

FileNotFoundError: [Errno 2] No such file or directory: 'NaiveBayesSorted.pkl'

In [437]:
# buatlah model naive bayes dengan sckit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Train the model
model = GaussianNB()

model.fit(X_train_sorted, y_train_sorted['attack_cat'])

prediction = model.predict(X_val_sorted)
accuracy = accuracy_score(y_val_sorted, prediction)
print("Accuracy: ", accuracy)

Accuracy:  0.644718033021228


## C. ID3

In [ ]:
from src.models.ID3 import ID3Algorithm

id3_model = ID3Algorithm()
id3_model.fit(X_train_sorted, y_train_sorted["attack_cat"])
id3_model.save_model("ID3Model.pkl")

KeyboardInterrupt: 

In [260]:
id3_model = ID3Algorithm.load_model("ID3Model.pkl")
id3_val_prediction = id3_model.predict(X_val_sorted)
accuracy_id3 = accuracy_score(y_val_sorted, id3_val_prediction)
print("ID3 Accuracy: ", accuracy_id3)

185360
374640
Model loaded from NaiveBayes.pkl
Accuracy:  0.2


## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [261]:

# Write your code here

# separated because deleting duplicate doesnt applied to test_set
duplicateImputer = DuplicateImputer(id_column='id', keep='first')
train_without_duplicate = duplicateImputer.fit_transform(train)

# cleaning
submission_pipe = Pipeline([("outlier", OutlierImputer()),
                 ("featureimputer", FeatureImputer()),
                 ("featureencoder", FeatureEncoder()),
                 ("imputer", FeatureScaler()),
                 ("pcaimputer", PCAImputer()),
                 ("normalizedimputer", FeatureScaler()),
                 ])

# splitting train to x and y
X_train_submission = train_without_duplicate.drop(['label','attack_cat','id'], axis=1)
y_train_submission = train_without_duplicate[['label','attack_cat']]
submission_test = test.drop('id', axis=1)

x_submission_train_preprocessed = submission_pipe.fit_transform(X_train_submission)
submission_test_preprocessed = submission_pipe.transform(submission_test)

# balancing x train
# separated from pipe because test doesnt get balanced
submission_imbalance_handler = ImbalancedDataHandler()
x_submission_train_balanced, y_submission_train_balanced = submission_imbalance_handler.fit_transform(x_submission_train_preprocessed, y_train_submission)

# variable to use in retrain and predict
x_submission_train_final = x_submission_train_balanced
y_submission_train_final = y_submission_train_balanced
train_submission_final = submission_test_preprocessed

# retrain model 

# predict


# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`

Drop missing data akan lebih baik dilakukan jika column-column yang mayoritasnya kosong sudah dilakukan. Namun, pada tugas kali ini kami tidak menghapusnya melainkan hanya menghapus kolom yang mayoritasnya kosong kemudian menghapus mengisi sisanya dengan nilai modus dan mean

Pada awalnya, data yang digunakan untuk train model adalah data yang di balanced dengan oversampling. Namun, hal ini justru menyebabkan pelatihan model yang lebih lama dan akurasinya pun tidak terlalu baik. Akurasi yang rendah ini dapat disebabkan oleh penyamarataan data jumlah data untuk setiap nilai target. Padahal, padatnya jumlah target dengan nilai tertentu mungkin terjadi karena kelas target, atau dalam kasus ini attack category, tersebut memang lebih sering terjadi. Oversampling rasanya juga kurang tepat mengingat jumlah data yang cukup besar untuk variasi nilai target yang tidak terlalu banyak.

Algoritma ID3 cenderung melakukan proses lebih lama, hal ini disebabkan oleh proses fitting nya yang perlu melakukan operasi yang cukup berat seperti menghitung information gain, entropy, dll dari tiap feature.
Algoritma Naive Bayes relatif cepat karena tidak perlu membangun tree dan operasi yang rumit. Secara keseluruhan hanya operasi kali bagi dan tambah sehingga memerlukan power processing yang lebih kecil. 
